In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 360, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 360 (delta 70), reused 51 (delta 51), pack-reused 269
Receiving objects: 100% (360/360), 100.20 KiB | 8.35 MiB/s, done.
Resolving deltas: 100% (172/172), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 6.4 MB/s eta 0:00:00
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
We will now install RAPIDS cuDF, cuML, and cuGraph via pip! 
Please stand by, should be quick...
***********************************************************************

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.0/

In [1]:
import cudf, cuml, cupy
from cudf.core.dataframe import DataFrame as cu_df
from cudf.core.series import Series as cu_series

/usr/local/lib/python3.9/dist-packages/rmm/__init__.py:56: FutureWarning: Use of 'rmm.RMMNumbaManager' is deprecated and will be removed. 'RMMNumbaManager' now lives in the 'rmm.allocators.numba' sub-module, please update your imports.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/rmm/__init__.py:56: FutureWarning: Use of 'rmm.rmm_cupy_allocator' is deprecated and will be removed. 'rmm_cupy_allocator' now lives in the 'rmm.allocators.cupy' sub-module, please update your imports.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/rmm/__init__.py:56: FutureWarning: Use of 'rmm.rmm_cupy_allocator' is deprecated and will be removed. 'rmm_cupy_allocator' now lives in the 'rmm.allocators.cupy' sub-module, please update your imports.
  warnings.warn(


In [2]:
import numpy as np
import pandas as pd
import os
from functools import partial
import scipy as sp
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score
import xgboost as xgb
import random

In [3]:
SEED = 29
NUM_FOLDS = 10

In [4]:
def seed_everything(seed=SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(seed=SEED)

In [5]:
train = pd.read_csv('/content/drive/MyDrive/Colab/2303_PGS3E5/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab/2303_PGS3E5/test.csv')
sample = pd.read_csv('/content/drive/MyDrive/Colab/2303_PGS3E5/sample_submission.csv')

In [ ]:
train.describe()

,Id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,2056.000000,2056.000000,2056.000000,2056.000000,2056.000000,2056.000000,2056.000000,2056.000000,2056.000000,2056.000000,2056.000000,2056.000000,2056.000000
mean,1027.500000,8.365175,0.527601,0.265058,2.398881,0.081856,16.955982,49.236868,0.996748,3.310569,0.641308,10.414972,5.720817
std,593.660397,1.705390,0.173164,0.188267,0.858824,0.023729,10.009710,32.961141,0.001827,0.142321,0.137942,1.028825,0.853146
min,0.000000,5.000000,0.180000,0.000000,1.200000,0.012000,1.000000,7.000000,0.990070,2.740000,0.390000,8.700000,3.000000
25%,513.750000,7.200000,0.390000,0.090000,1.900000,0.071000,8.000000,22.000000,0.995600,3.200000,0.550000,9.500000,5.000000
50%,1027.500000,7.950000,0.520000,0.250000,2.200000,0.079000,16.000000,44.000000,0.996700,3.310000,0.610000,10.100000,6.000000
75%,1541.250000,9.200000,0.640000,0.420000,2.600000,0.090000,24.000000,65.000000,0.997800,3.390000,0.720000,11.000000,6.000000
max,2055.000000,15.900000,1.580000,0.760000,14.000000,0.414000,68.000000,289.000000,1.003690,3.780000,1.950000,14.000000,8.000000


In [6]:
train['total_acid'] = train['fixed acidity'] + train['volatile acidity'] + train['citric acid']
train['acid/density'] = train['total_acid']  / train['density']
train['alcohol_density'] = train['alcohol']  * train['density']
train['sulphate/density'] = train['sulphates']  / train['density']
train['sulphates/acid'] = train['sulphates'] / train['volatile acidity']
train['sulphates/chlorides'] = train['sulphates'] / train['chlorides']
train['sulphates*alcohol'] = train['sulphates'] * train['alcohol']

test['total_acid'] = test['fixed acidity'] + test['volatile acidity'] + test['citric acid']
test['acid/density'] = test['total_acid']  / test['density']
test['alcohol_density'] = test['alcohol']  * test['density']
test['sulphate/density'] = test['sulphates']  / test['density']
test['sulphates/acid'] = test['sulphates'] / test['volatile acidity']
test['sulphates/chlorides'] = test['sulphates'] / test['chlorides']
test['sulphates*alcohol'] = test['sulphates'] * test['alcohol']

In [7]:
X = train.drop(columns=['quality', 'Id'])
features = X.columns
X_test = test.drop(columns=['Id'])
y = train.quality

In [8]:
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 3
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 4
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 5
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 6
            elif pred >= coef[3] and pred < coef[4]:
                X_p[i] = 7
            else:
                X_p[i] = 8

        ll = cohen_kappa_score(y, X_p, weights='quadratic')
        return -ll

    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [3.5, 4.5, 5.5, 6.5, 7.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')

    def predict(self, X, coef):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 3
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 4
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 5
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 6
            elif pred >= coef[3] and pred < coef[4]:
                X_p[i] = 7
            else:
                X_p[i] = 8
        return X_p.astype('int')

    def coefficients(self):
        return self.coef_['x']

In [9]:
def cross_valid(model, train, target, test, num_folds=10, random_state=42):

    train_oof = np.zeros((len(train)))
    test_preds = 0

    kf = StratifiedKFold(n_splits=num_folds, random_state=SEED, shuffle=True)
    scores = []
    
    params = {
        'random_state': SEED,
        'nthread': -1,
        'objective': 'reg:squarederror',


        #'lambda': 8.59836446748171, 'alpha': 4.567193355350751, 'max_depth': 17, 'eta': 0.23458205995850612, 'gamma': 4.807997278560724, 'min_child_weight': 9, 'subsample': 0.8253475666406669, 'colsample_bytree': 0.6943356728816786, 'max_delta_step': 69.00108273635568, 'num_boost_round': 700, 'grow_policy': 'lossguide'
        'lambda': 3.672657391606741, 'alpha': 7.946537191464724, 'max_depth': 37, 'eta': 0.10757388518205632, 'gamma': 2.098019969954558, 'min_child_weight': 35, 'subsample': 0.7849016222932254, 'max_delta_step': 39.125794484854595, 'num_boost_round': 1800, 'grow_policy': 'lossguide'
     }
    
    num_rounds = params['num_boost_round']
    
    xgb_train_preds = np.zeros(len(train.index), )
    
    coef = []
    

    for f, (train_ind, val_ind) in enumerate(kf.split(train, target)):

        train_df, val_df = train.iloc[train_ind][columns], train.iloc[val_ind][columns]
        
        train_target, val_target = target[train_ind], target[val_ind]
        

        xgb_x_train = pd.DataFrame(train_df)
        xgb_x_valid = pd.DataFrame(val_df)

        xgb_x_train_cudf = cu_df(xgb_x_train)
        y_train_cudf = cu_series(train_target)
        xgb_x_valid_cudf = cu_df(xgb_x_valid)
        y_valid_cudf = cu_series(val_target)

        trn_data = xgb.DMatrix(xgb_x_train_cudf, label=y_train_cudf)
        val_data = xgb.DMatrix(xgb_x_valid_cudf, label=y_valid_cudf)

        model = xgb.train(params, 
        trn_data,
        num_rounds,
        evals = [(val_data, "val_data")], 
        verbose_eval=False, 
        early_stopping_rounds=50
        )

        xgb_valid_preds = model.predict(xgb.DMatrix(xgb_x_valid_cudf), iteration_range = (0, model.best_iteration+1))
        
        optR = OptimizedRounder()
        optR.fit(xgb_valid_preds, val_target)
                
        temp_oof = optR.predict(xgb_valid_preds, optR.coefficients())
        
        train_oof[val_ind] = temp_oof
    
        test_oof_preds = model.predict(xgb.DMatrix(test[columns]), iteration_range = (0, model.best_iteration+1))
        
        test_oof_preds = optR.predict(test_oof_preds, optR.coefficients())
        
        coef.append(optR.coefficients())
        
        print(optR.coefficients())
        
        test_preds += test_oof_preds/num_folds
        
        scores.append(cohen_kappa_score(val_target, temp_oof, weights='quadratic'))
        
        print("Fold " , f, " ", cohen_kappa_score(val_target, temp_oof, weights="quadratic"))
        
                                
    print("Mean Kappa Score: ", np.mean(scores))
    print("Kappa Score OOF: ", cohen_kappa_score(y, train_oof, weights='quadratic'))

    return train_oof, test_preds, np.mean(scores), coef

In [10]:
## Here I can select which features I want to use
columns = features

## If you want to do ensemble is good to save the oof train / preds
train_oof_1, test_preds_1, score_oof_1, coef = cross_valid(None, X, y, X_test, num_folds=NUM_FOLDS, random_state=SEED)

[06:50:03] WARNING: ../src/learner.cc:767: 
Parameters: { "num_boost_round" } are not used.

[3.44709988 4.81338069 5.5474565  6.02563569 7.18379299]
Fold  0   0.6386656461283327
[06:50:04] WARNING: ../src/learner.cc:767: 
Parameters: { "num_boost_round" } are not used.

[3.66133389 4.62824991 5.7244354  6.25413083 7.02448409]
Fold  1   0.6154704999825547
[06:50:05] WARNING: ../src/learner.cc:767: 
Parameters: { "num_boost_round" } are not used.

[3.58293017 4.6150544  5.64663854 6.03170707 7.60305211]
Fold  2   0.5650721623495304
[06:50:06] WARNING: ../src/learner.cc:767: 
Parameters: { "num_boost_round" } are not used.

[3.61399402 4.5920981  5.47735955 6.30272949 7.39676828]
Fold  3   0.605045408678103
[06:50:07] WARNING: ../src/learner.cc:767: 
Parameters: { "num_boost_round" } are not used.

[3.57826786 4.59001262 5.53412272 6.17658849 7.63912192]
Fold  4   0.6200834223861561
[06:50:09] WARNING: ../src/learner.cc:767: 
Parameters: { "num_boost_round" } are not used.

[3.59319981 4

In [12]:
sample['quality'] = test_preds_1.round().astype(int)
sample.to_csv('/content/drive/MyDrive/Colab/2303_PGS3E5/submission.csv', index=False)
sample

,Id,quality
0,2056,5
1,2057,6
2,2058,5
3,2059,6
4,2060,6
...,...,...
1367,3423,6
1368,3424,7
1369,3425,5
1370,3426,5


# Xgb params tuning

In [13]:
!pip -q install optuna

In [10]:
import optuna
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

def objective(trial, train=X, target=y):

    params = {
        'random_state': SEED,
        'nthread': -1, 
        'objective': 'reg:squarederror',

        'lambda': trial.suggest_float('lambda', 1e-8, 10),
        'alpha': trial.suggest_float('alpha', 1e-8, 10),
        'max_depth': trial.suggest_int('max_depth',2, 50),
        'eta': trial.suggest_float("eta", 1e-8, 1.0),
        'gamma': trial.suggest_float("gamma", 1e-8, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 50),
        "subsample": trial.suggest_float("subsample", 0, 1.0),
        #"colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
        'max_delta_step': trial.suggest_float("max_delta_step", 0, 100),
        'num_boost_round': trial.suggest_int('num_boost_round', 500, 2000, 100),
        'grow_policy': trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"]),
        
    }
    num_rounds = params['num_boost_round']

    X_train, X_valid, y_train, y_valid = train_test_split(train, target, test_size=0.4)

    trn_data = xgb.DMatrix(X_train, label=y_train)
    val_data = xgb.DMatrix(X_valid, label=y_valid)

    model = xgb.train(params, 
    trn_data,
    num_rounds,
    evals = [(val_data, "val_data")], 
    verbose_eval=False, 
    early_stopping_rounds=50
    )

    xgb_valid_preds = model.predict(xgb.DMatrix(X_valid), iteration_range = (0, model.best_iteration+1))
        
    optR = OptimizedRounder()
    optR.fit(xgb_valid_preds, y_valid)

    xgb_valid_preds = optR.predict(xgb_valid_preds, optR.coefficients())
                


    return cohen_kappa_score(y_valid, xgb_valid_preds, weights="quadratic")

# Creating Optuna object and defining its parameters
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials = 1000)

# Showing optimization results
print('Number of finished trials:', len(study.trials))
print('Best trial parameters:', study.best_trial.params)
print('Best score:', study.best_value)

[I 2023-03-22 06:01:13,384] A new study created in memory with name: no-name-f049718f-54c1-42ac-9b68-71d99d69dc62


[06:01:14] WARNING: ../src/learner.cc:767: 
Parameters: { "num_boost_round" } are not used.



[I 2023-03-22 06:01:19,001] Trial 0 finished with value: 0.547383624157712 and parameters: {'lambda': 2.6635993038774775, 'alpha': 0.6804732346124048, 'max_depth': 35, 'eta': 0.18070312810027347, 'gamma': 0.5721966331755872, 'min_child_weight': 17, 'subsample': 0.2448022307397919, 'max_delta_step': 90.07834475206396, 'num_boost_round': 1700, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 0.547383624157712.


[06:01:19] WARNING: ../src/learner.cc:767: 
Parameters: { "num_boost_round" } are not used.



[I 2023-03-22 06:01:21,170] Trial 1 finished with value: 0.5546243116418748 and parameters: {'lambda': 3.161064860648548, 'alpha': 2.770566720395379, 'max_depth': 25, 'eta': 0.0831680334765412, 'gamma': 1.8341715683496564, 'min_child_weight': 27, 'subsample': 0.5820772481478932, 'max_delta_step': 31.915711887552455, 'num_boost_round': 700, 'grow_policy': 'lossguide'}. Best is trial 1 with value: 0.5546243116418748.


...

[06:43:26] WARNING: ../src/learner.cc:767: 
Parameters: { "num_boost_round" } are not used.



[I 2023-03-22 06:43:28,423] Trial 998 finished with value: 0.5355993542625249 and parameters: {'lambda': 9.0420480453555, 'alpha': 9.30606673040898, 'max_depth': 33, 'eta': 0.6554924405836173, 'gamma': 6.323219355208902, 'min_child_weight': 38, 'subsample': 0.7943616532992073, 'max_delta_step': 56.8070031351032, 'num_boost_round': 1700, 'grow_policy': 'depthwise'}. Best is trial 726 with value: 0.6210790436600513.


[06:43:28] WARNING: ../src/learner.cc:767: 
Parameters: { "num_boost_round" } are not used.



[I 2023-03-22 06:43:30,628] Trial 999 finished with value: 0.5607980458237211 and parameters: {'lambda': 7.298298053302066, 'alpha': 7.727317914905242, 'max_depth': 29, 'eta': 0.03344461221502241, 'gamma': 2.6552637965751176, 'min_child_weight': 35, 'subsample': 0.37778166642951794, 'max_delta_step': 1.4122366326279305, 'num_boost_round': 1900, 'grow_policy': 'depthwise'}. Best is trial 726 with value: 0.6210790436600513.


Number of finished trials: 1000
Best trial parameters: {'lambda': 3.672657391606741, 'alpha': 7.946537191464724, 'max_depth': 37, 'eta': 0.10757388518205632, 'gamma': 2.098019969954558, 'min_child_weight': 35, 'subsample': 0.7849016222932254, 'max_delta_step': 39.125794484854595, 'num_boost_round': 1800, 'grow_policy': 'lossguide'}
Best score: 0.6210790436600513


In [ ]:
from optuna.visualization import plot_optimization_history

plotly_config = {"staticPlot": True}

fig = plot_optimization_history(study)
fig.show(config=plotly_config)

In [ ]:
from optuna.visualization import plot_param_importances

fig = plot_param_importances(study)
fig.show(config=plotly_config)